#Credit Risk
pada latihan ini, kita akan mencoba memembuat model untuk memprediksi mana nasabah yang akan gagal bayar dan yang tidak (jenis problemnya 'Binary Classification'). Data  yang digunakan cukup besar, sehingga bisa dibilang ini sebagai latihan dalam memproses Big Data.

Tujuan utamanya adalah membuat model prediksi terkait nasabah yang akan Gagal Bayar dan yang tidak.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# karena filenya cukup berat, jadi kita aplod aja filenya ke sini (ga pake spreadsheet)
!dir

drive  sample_data


In [ ]:
# bikin variable dulu buat ngebaca dataframenya
# df = pd.read_csv("lc_2016_2017.csv")
# df.head()

Cara Upload File CSV dari GDrive

In [ ]:
# import dulu librarynya
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cr = pd.read_csv('/content/drive/MyDrive/Belajar DA & DS/Kumpulan projek portofolio/Credit Risk/lc_2016_2017.csv')
cr.head()

<ipython-input-112-87d2bd25bd86>:1: DtypeWarning: Columns (18,54) have mixed types. Specify dtype option on import or set low_memory=False.
  cr = pd.read_csv('/content/drive/MyDrive/Belajar DA & DS/Kumpulan projek portofolio/Credit Risk/lc_2016_2017.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,112435993,NaN,2300,2300,2300.0,36 months,12.62,77.08,C,C1,...,0.0,NaN,1.0,2.0,2315.0,55.0,7100.0,1.0,0.0,2.0
1,112290210,NaN,16000,16000,16000.0,60 months,12.62,360.95,C,C1,...,11078.0,69.0,3.0,5.0,1962.0,94.0,31900.0,0.0,6.0,1.0
2,112436985,NaN,6025,6025,6025.0,36 months,15.05,209.01,C,C4,...,0.0,NaN,1.0,2.0,1950.0,45.0,27700.0,1.0,5.0,3.0
3,112439006,NaN,20400,20400,20400.0,36 months,9.44,652.91,B,B1,...,53566.0,95.0,1.0,2.0,4240.0,60.0,46900.0,1.0,1.0,1.0
4,112438929,NaN,13000,13000,13000.0,36 months,11.99,431.73,B,B5,...,8466.0,72.0,0.0,1.0,2996.0,78.0,7800.0,0.0,0.0,0.0


In [ ]:
cr.shape

(759338, 72)

In [ ]:
cr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759338 entries, 0 to 759337
Data columns (total 72 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           759338 non-null  int64  
 1   member_id                    0 non-null       float64
 2   loan_amnt                    759338 non-null  int64  
 3   funded_amnt                  759338 non-null  int64  
 4   funded_amnt_inv              759338 non-null  float64
 5   term                         759338 non-null  object 
 6   int_rate                     759338 non-null  float64
 7   installment                  759338 non-null  float64
 8   grade                        759338 non-null  object 
 9   sub_grade                    759338 non-null  object 
 10  emp_title                    708368 non-null  object 
 11  emp_length                   708975 non-null  object 
 12  home_ownership               759338 non-null  object 
 13 

ada beberapa feature yang memiliki data null yang sangat banyak, seperti member_id, desc dan lain2nya.

Target Variable

Dikarenakan project ini untuk mengetahui bad loan (nasabah buruk) & good loan (nasabah baik), maka perlu dibuat feature baru, yaitu target variable yang merepresentasikan bad loan (sebagai 1) dan good loan (sebagai 0)

In [ ]:
# Melihat unique values pada feature loan_status
cr.loan_status.unique()

array(['Current', 'Fully Paid', 'In Grace Period', 'Late (31-120 days)',
       'Late (16-30 days)', 'Charged Off', 'Default'], dtype=object)

In [ ]:
# kita klasifikasikan, yang Bad Loan itu yang

# Membuat feature baru yaitu good_bad sebagai target variable,
# Jika loan_statusnya 'Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)'
# akan dianggap sebagai bad_loan atau 1 dan nilai selain itu akan dianggap good loan atau 0

def good_bad(row):
    if row['loan_status'] in ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)']:
        return 1
    else:
        return 0

In [ ]:
cr['good_bad'] = cr.apply(good_bad, axis=1)
cr.good_bad.value_counts()

,count
good_bad,
0,702875
1,56463


In [ ]:
# cek distribusinya, normal apa ngga
cr.good_bad.value_counts(normalize=True)

,proportion
good_bad,
0,0.925642
1,0.074358


ternyata hasilnya bisa kita bilang jomplang (imbalance) yang artinya ga seimbang. data mayoritas berada disekitar nol. uuntuk lebih rincinya, kita perlu cek probabilitas dari tiap datanya. sebelum itu, kita perlu cleaning data dan pilih feature apa saja yang akan digunakan

##Missing Value
karena pada sebelumnya kita tahu bahwa ada beberapa feature yang memiliki missing value yang sangat banyak.
Maka dari itu, feature yang memiliki missing values lebih dari 50% akan di drop, karena jika ingin diisi dengan nilai lain seperti median atau mean, maka errornya akan sangat tinggi. Lebih baik di drop agar tidak membuat model semakin tidak akurat.

In [ ]:
cr.isnull().sum()

,0
id,0
member_id,759338
loan_amnt,0
funded_amnt,0
funded_amnt_inv,0
...,...
total_rev_hi_lim,0
inq_fi,61
total_cu_tl,62
inq_last_12m,62


In [ ]:
# Melihat feature apa saja yang memiliki missing value lebih dari 50%
missing_values = pd.DataFrame(cr.isnull().sum()/cr.shape[0])
missing_values = missing_values[missing_values.iloc[:,0] > 0.50] #<---------- inilah yang mengatur 50% nya
missing_values.sort_values([0], ascending=False)

,0
member_id,1.000000
desc,0.999978
dti_joint,0.955223
annual_inc_joint,0.955220
verification_status_joint,0.955220
mths_since_last_record,0.814076
mths_since_last_major_derog,0.716720


In [ ]:
# Drop feature-feature tersebut
# cr.dropna(thresh = cr.shape[0]*0.5, how='all', axis=1, inplace=True) <------ bisa pake ini, tp sya kurang faham

In [ ]:
cr.drop(['member_id', 'desc', 'dti_joint', 'annual_inc_joint',
         'verification_status_joint', 'mths_since_last_record',
         'mths_since_last_major_derog'],
        axis=1, inplace=True)

In [ ]:
# kita cek, udah ilang atau belum
missing_values = pd.DataFrame(cr.isnull().sum()/cr.shape[0])
missing_values = missing_values[missing_values.iloc[:,0] > 0.50] #<---------- inilah yang mengatur 50% nya
missing_values.sort_values([0], ascending=False)

,0


## Tinjau Data (Balance atau ngga)

In [ ]:
cr.shape

(759338, 66)

In [ ]:
cr.good_bad.value_counts()

,count
good_bad,
0,702875
1,56463


ternyata hasilnya imbalance (data tidak seimbang)

In [ ]:
702875/56463

12.44841754777465

data bernilai 0 itu 12x lipatnya yang 1

solusinya, kita akan buat data tersebut supaya seimbang/mendekati seimbang dgn cara menduplikat data yang bernilai 1 sebanyak 12x salinan

In [ ]:
# kumpulkan data good_bad yang bernilai 1, lalu duplikat sebanyak 12x
m = cr[cr['good_bad'] == 1]

In [ ]:
# kita coba duplicate m menjadi 12x
m = pd.concat([m]*12, ignore_index=True)

In [ ]:
m

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,good_bad
0,111897551,20000,20000,20000.0,36 months,18.06,723.66,D,D2,Transportation manager,...,81.0,1.0,1.0,10039.0,100.0,11350.0,1.0,3.0,2.0,1
1,111871350,5000,5000,5000.0,36 months,10.42,162.33,B,B3,Painter,...,71.0,0.0,0.0,1346.0,77.0,2200.0,0.0,2.0,0.0,1
2,110261189,35000,35000,35000.0,60 months,15.05,833.57,C,C4,Sales Agent,...,79.0,2.0,5.0,24857.0,84.0,269800.0,0.0,1.0,6.0,1
3,112179814,15000,15000,15000.0,36 months,30.65,642.13,F,F4,Service Adivsor,...,87.0,3.0,3.0,10614.0,75.0,40900.0,1.0,1.0,9.0,1
4,111076730,11000,11000,11000.0,36 months,10.42,357.12,B,B3,Cdl truck driver,...,41.0,1.0,1.0,7325.0,57.0,12900.0,1.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677551,66442993,25000,25000,24800.0,36 months,18.99,916.28,E,E3,Dispatch Manager,...,NaN,NaN,NaN,NaN,NaN,11600.0,NaN,NaN,NaN,1
677552,66492137,5000,5000,5000.0,36 months,15.41,174.34,D,D1,Process Control Inspector,...,NaN,NaN,NaN,NaN,NaN,10200.0,NaN,NaN,NaN,1
677553,65855145,6000,6000,6000.0,36 months,26.77,244.22,G,G1,NaN,...,NaN,NaN,NaN,NaN,NaN,2400.0,NaN,NaN,NaN,1
677554,66421505,24100,24100,24100.0,60 months,18.49,618.43,E,E2,supervisor,...,NaN,NaN,NaN,NaN,NaN,13400.0,NaN,NaN,NaN,1


In [ ]:
56463*12

677556

steleh selesai menduplikat data nilai 1, kemudian kita kumpulkan data good_bad yang bernilai 0, lalu gabungkan dgn yang bernilai 1 yang sudah di duplikat tadi.

In [ ]:
n = cr[cr['good_bad'] == 0]

In [ ]:
# gabungkan data m (yang bernilai 1 yang udh diduplikat) dan n (yang benrilai 0)
b = pd.concat([m,n])

# reset index supaya gada index yang bertumpuk
b.reset_index(drop=True, inplace=True)

In [ ]:
b.good_bad.value_counts()

,count
good_bad,
0,702875
1,677556


In [ ]:
b.shape

(1380431, 66)

In [ ]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1380431 entries, 0 to 1380430
Data columns (total 66 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   id                          1380431 non-null  int64  
 1   loan_amnt                   1380431 non-null  int64  
 2   funded_amnt                 1380431 non-null  int64  
 3   funded_amnt_inv             1380431 non-null  float64
 4   term                        1380431 non-null  object 
 5   int_rate                    1380431 non-null  float64
 6   installment                 1380431 non-null  float64
 7   grade                       1380431 non-null  object 
 8   sub_grade                   1380431 non-null  object 
 9   emp_title                   1276606 non-null  object 
 10  emp_length                  1277609 non-null  object 
 11  home_ownership              1380431 non-null  object 
 12  annual_inc                  1380431 non-null  float64
 1

##Data Splitting
supaya lebih efisien, kita split dulu data nya diawal, baru lakukan cleaning data.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Membagi data menjadi 80/20 dengan menyamakan distribusi dari bad loans di data test dengan data train.
x = b.drop('good_bad', axis=1)
y = b['good_bad']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify= y, random_state=42)

In [ ]:
x.shape

(1380431, 65)

In [ ]:
# kita coba cek, apakah distribusinya sama antara y_train dengan y_test
y_train.value_counts(normalize=True)

,proportion
good_bad,
0,0.509171
1,0.490829


In [ ]:
y_test.value_counts(normalize=True)

,proportion
good_bad,
0,0.509169
1,0.490831


Distribusi y_test sudah mendekati sama dengan y_train.

##Data Cleaning
pada proses ini, mengingat data Credit Risk yang sangat besar, kemungkinan akan sangat repot jika dikerjakan satu-satu. Maka, kita akan coba langsung cek semuanya, tapi difilter berdasarkan tipenya. (numeric dan categoric)

In [ ]:
for col in x_train.select_dtypes(include= ['object','bool']).columns:
  print(col)
  print(x_train[col].unique())
  print()

term
['36 months' '60 months']

grade
['D' 'C' 'B' 'E' 'F' 'A' 'G']

sub_grade
['D1' 'C5' 'B5' 'B1' 'C1' 'C4' 'D3' 'C2' 'C3' 'E4' 'E1' 'B2' 'D2' 'F1'
 'F2' 'F5' 'E2' 'B3' 'A5' 'B4' 'G3' 'A3' 'D4' 'D5' 'A1' 'A4' 'F3' 'A2'
 'E5' 'F4' 'E3' 'G5' 'G1' 'G4' 'G2']

emp_title
['Yard maintenance' 'Tech 1 maintenance' 'Teacher' ...
 'Vendor Operations Manager' 'drain cleaner' 'Hourly Manager']

emp_length
['5 years' '< 1 year' '10+ years' '6 years' '3 years' '1 year' nan
 '4 years' '8 years' '9 years' '2 years' '7 years']

home_ownership
['RENT' 'MORTGAGE' 'OWN' 'ANY' 'NONE']

verification_status
['Source Verified' 'Not Verified' 'Verified']

issue_d
['Jan-2017' 'May-2017' 'Oct-2016' 'Mar-2016' 'Aug-2017' 'Sep-2017'
 'Aug-2016' 'Jan-2016' 'Apr-2017' 'Jun-2017' 'Feb-2017' 'Mar-2017'
 'Jul-2017' 'Jul-2016' 'Sep-2016' 'Jun-2016' 'Feb-2016' 'Apr-2016'
 'Dec-2016' 'May-2016' 'Nov-2016']

loan_status
['Charged Off' 'Current' 'Fully Paid' 'Late (31-120 days)'
 'Late (16-30 days)' 'In Grace Period' 'Def

In [ ]:
# kesimpulannya, berikut kolom/feature yang harus di cleaning
col_need_to_clean = ['term', 'emp_length', 'issue_d', 'earliest_cr_line', 'last_pymnt_d',
                    'next_pymnt_d', 'last_credit_pull_d']

###kolom 'term'

In [ ]:
# Menghilangkan ' months' menjadi '' (ngapus keterangan months nya)
x_train['term'].str.replace(' months', '')

,term
345211,36
741379,36
1248925,36
610328,36
187906,36
...,...
24027,36
458450,60
1292095,36
1028377,60


In [ ]:
# Convert data type menjadi numeric
x_train['term'] = pd.to_numeric(x_train['term'].str.replace(' months', ''))

In [ ]:
# lakukan hal yang sama pada x_test
x_test['term'].str.replace(' months', '')
x_test['term'] = pd.to_numeric(x_test['term'].str.replace(' months', ''))

In [ ]:
x_test.term.head()

,term
127029,36
236567,36
183902,36
126091,36
1335086,60


###kolom 'emp_length

In [ ]:
# Cek values apa saja yang harus di cleaning
x_train['emp_length'].unique()

array(['5 years', '< 1 year', '10+ years', '6 years', '3 years', '1 year',
       nan, '4 years', '8 years', '9 years', '2 years', '7 years'],
      dtype=object)

In [ ]:
# kita hapusin keterangan tahun dan lain-lainnya, supaya hanya tersisa angka
x_train['emp_length'] = x_train['emp_length'].str.replace('\+', '', regex=True) # kegunaan regex=True adalah untuk nyocokin pola
x_train['emp_length'] = x_train['emp_length'].str.replace(r'\D', '', regex=True)  # Remove semua karakter non-digit

# Cara lebih detailnya, bisa kita ubah satu-satu.
# x_train['emp_length'] = x_train['emp_length'].str.replace("\+ years", '')
# x_train['emp_length'] = x_train['emp_length'].str.replace(' years', '')
# x_train['emp_length'] = x_train['emp_length'].str.replace('< 1 year', str(0))
# x_train['emp_length'] = x_train['emp_length'].str.replace(' year', '')


x_train['emp_length'].fillna(value = 0, inplace=True)
x_train['emp_length'] = pd.to_numeric(x_train['emp_length'])

<ipython-input-148-0d8e5b6dc43e>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_train['emp_length'].fillna(value = 0, inplace=True)


In [ ]:
x_train.emp_length

,emp_length
345211,5
741379,1
1248925,10
610328,6
187906,5
...,...
24027,10
458450,10
1292095,10
1028377,7


In [ ]:
# lakukan hal yang sama ke x_test
x_test['emp_length'] = x_test['emp_length'].str.replace('\+', '', regex=True) # kegunaan regex=True adalah untuk nyocokin pola
x_test['emp_length'] = x_test['emp_length'].str.replace(r'\D', '', regex=True)  # Remove semua karakter non-digit

x_test['emp_length'].fillna(value = 0, inplace=True)
x_test['emp_length'] = pd.to_numeric(x_test['emp_length'])

<ipython-input-150-461b94d91fda>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_test['emp_length'].fillna(value = 0, inplace=True)


In [ ]:
x_test.emp_length

,emp_length
127029,2
236567,1
183902,0
126091,6
1335086,10
...,...
800934,3
87062,10
925065,10
538286,9


###kolom 'Date'
berikut beberapa kolom tentang date :
1. 'issue_d',
2. 'earliest_cr_line',
3. 'last_pymnt_d',
4. 'next_pymnt_d',
5. 'last_credit_pull_d'


In [ ]:
# Cek feature date
col_date = ['issue_d', 'earliest_cr_line', 'last_pymnt_d',
                    'next_pymnt_d', 'last_credit_pull_d']

x_train[col_date]

,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
345211,Jan-2017,May-2007,Mar-2017,NaN,Sep-2017
741379,May-2017,Dec-2007,Dec-2017,Jan-2018,Dec-2017
1248925,Oct-2016,Jan-2001,May-2017,NaN,Dec-2017
610328,Mar-2016,Dec-2002,Aug-2016,NaN,Dec-2017
187906,Aug-2017,Apr-2009,NaN,Jan-2018,Dec-2017
...,...,...,...,...,...
24027,May-2016,Sep-1997,Aug-2017,NaN,Dec-2017
458450,Jan-2017,Oct-1986,Sep-2017,Jan-2018,Dec-2017
1292095,Mar-2016,Jul-2000,Dec-2017,Jan-2018,Dec-2017
1028377,Aug-2017,Apr-1996,Dec-2017,Jan-2018,Dec-2017


In [ ]:
x_train[col_date].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1104344 entries, 345211 to 82242
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   issue_d             1104344 non-null  object
 1   earliest_cr_line    1104344 non-null  object
 2   last_pymnt_d        1095272 non-null  object
 3   next_pymnt_d        642908 non-null   object
 4   last_credit_pull_d  1104318 non-null  object
dtypes: object(5)
memory usage: 50.6+ MB


krna masi objek, kita ubah ke Datetime

In [ ]:
for col in col_date:
    x_train[col] = pd.to_datetime(x_train[col])

<ipython-input-154-144a995d7323>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_train[col] = pd.to_datetime(x_train[col])
<ipython-input-154-144a995d7323>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_train[col] = pd.to_datetime(x_train[col])
<ipython-input-154-144a995d7323>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_train[col] = pd.to_datetime(x_train[col])
<ipython-input-154-144a995d7323>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please 

In [ ]:
x_train[col_date].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1104344 entries, 345211 to 82242
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   issue_d             1104344 non-null  datetime64[ns]
 1   earliest_cr_line    1104344 non-null  datetime64[ns]
 2   last_pymnt_d        1095272 non-null  datetime64[ns]
 3   next_pymnt_d        642908 non-null   datetime64[ns]
 4   last_credit_pull_d  1104318 non-null  datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 50.6 MB


In [ ]:
# lakukan hal yang sama pada x_test
for col in col_date:
    x_test[col] = pd.to_datetime(x_test[col])

<ipython-input-156-a918e8e403a1>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_test[col] = pd.to_datetime(x_test[col])
<ipython-input-156-a918e8e403a1>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_test[col] = pd.to_datetime(x_test[col])
<ipython-input-156-a918e8e403a1>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_test[col] = pd.to_datetime(x_test[col])
<ipython-input-156-a918e8e403a1>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specif

In [ ]:
x_test[col_date].info()

<class 'pandas.core.frame.DataFrame'>
Index: 276087 entries, 127029 to 44845
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   issue_d             276087 non-null  datetime64[ns]
 1   earliest_cr_line    276087 non-null  datetime64[ns]
 2   last_pymnt_d        273783 non-null  datetime64[ns]
 3   next_pymnt_d        160441 non-null  datetime64[ns]
 4   last_credit_pull_d  276084 non-null  datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 12.6 MB


In [ ]:
# Check apakah berhasil di cleaning
x_test[col_need_to_clean].info()

<class 'pandas.core.frame.DataFrame'>
Index: 276087 entries, 127029 to 44845
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   term                276087 non-null  int64         
 1   emp_length          276087 non-null  int64         
 2   issue_d             276087 non-null  datetime64[ns]
 3   earliest_cr_line    276087 non-null  datetime64[ns]
 4   last_pymnt_d        273783 non-null  datetime64[ns]
 5   next_pymnt_d        160441 non-null  datetime64[ns]
 6   last_credit_pull_d  276084 non-null  datetime64[ns]
dtypes: datetime64[ns](5), int64(2)
memory usage: 16.9 MB


oke, sekarang semua kolom yang kotor sebelumnya udah bersih (sesuai dengan keperluan modelling)

##Feature Engineering
pada tahap ini, kita akan sedikit merubah/menambah fitur data nya supaya bisa lebih akurat nanti modelnya

kita akan buat function terkait berapa lama nasabah tersebut nunggaknya, atau berapa lama lagi cicilan nasabah tersebut smpe lunasnya

In [ ]:
# Kolom yang akan di feature engineering
col_need_to_clean

['term',
 'emp_length',
 'issue_d',
 'earliest_cr_line',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d']

In [ ]:
X_train = x_train[col_need_to_clean]
X_test = x_test[col_need_to_clean]

X_train.head()

,term,emp_length,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
345211,36,5,2017-01-01,2007-05-01,2017-03-01,NaT,2017-09-01
741379,36,1,2017-05-01,2007-12-01,2017-12-01,2018-01-01,2017-12-01
1248925,36,10,2016-10-01,2001-01-01,2017-05-01,NaT,2017-12-01
610328,36,6,2016-03-01,2002-12-01,2016-08-01,NaT,2017-12-01
187906,36,5,2017-08-01,2009-04-01,NaT,2018-01-01,2017-12-01


In [ ]:
X_test.head()

,term,emp_length,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
127029,36,2,2016-08-01,2004-12-01,2017-08-01,2018-01-01,2017-12-01
236567,36,1,2016-08-01,1998-09-01,2017-10-01,2018-01-01,2017-12-01
183902,36,0,2016-07-01,2001-02-01,2017-05-01,NaT,2017-11-01
126091,36,6,2016-08-01,2004-08-01,2016-10-01,NaT,2017-03-01
1335086,60,10,2016-02-01,1995-08-01,2017-12-01,2018-01-01,2017-12-01


In [ ]:
# tidak dibutuhkan untuk feature engineering
del X_train['next_pymnt_d']
del X_test['next_pymnt_d']


In [ ]:
X_train.shape, X_test.shape

((1104344, 6), (276087, 6))

In [ ]:
from datetime import date

date.today().strftime('%Y-%m-%d') #<--------untuk liat skrg tanggal berapa

'2025-01-13'

In [ ]:
# kita coba lihat seperti apa data smentara kita
X_train.head()

,term,emp_length,issue_d,earliest_cr_line,last_pymnt_d,last_credit_pull_d
345211,36,5,2017-01-01,2007-05-01,2017-03-01,2017-09-01
741379,36,1,2017-05-01,2007-12-01,2017-12-01,2017-12-01
1248925,36,10,2016-10-01,2001-01-01,2017-05-01,2017-12-01
610328,36,6,2016-03-01,2002-12-01,2016-08-01,2017-12-01
187906,36,5,2017-08-01,2009-04-01,NaT,2017-12-01


In [ ]:
# kita coba untuk melihat durasi lamanya, contoh kita ambil dari kolom issue_d baris pertama
pd.to_datetime(date.today().strftime('%Y-%m-%d')) - pd.to_datetime('2017-02-01')


Timedelta('2903 days 00:00:00')

In [ ]:
2899/30

96.63333333333334

kesimpulannya, sudah selama 2899 hari (96 bulan) sejak dicairkan untuk pertama kali kepada si nasabah ini. ini termasuk sudah lumayan lama, apakah nasabah ini akan membayar atau tidak? nah, informasi ini akan sangat berguna untuk membuat akurasi model kita jadi lebih akurat.

ini juga berlaku untuk kolom lainnya, seperti earliest_cr_line, kolom yang menginformasikan kapan kartu kredit pertama kali diterbitkan. kita coba pada baris pertama earliest_cr_line


In [ ]:
pd.to_datetime(date.today().strftime('%Y-%m-%d')) - pd.to_datetime('1994-08-01')

Timedelta('11123 days 00:00:00')

nah, kesimpulannya berarti sudah sejak 11119 hari sejak kartu kredit pertama kali diterbitkan. nah, kita akan coba buat functionnya terlebih dulu supaya bisa di apply secara menyeluruh (ngga satu satu kya barusan)



In [ ]:
# feature engineering untuk kolom 'date'. tujuannya membuat kolom durasi (satuannya 'per bulan').
# supaya lebih mudah untuk diproses oleh modelnya nanti
def date_columns(df, column):
    today_date = pd.to_datetime(date.today().strftime('%Y-%m-%d')) #<--- mendefinisikan tanggal hari ini dgn format (yyyy-mm-dd)
    df[column] = pd.to_datetime(df[column], format = "%Y-%m-%d") #<---mendefinisikan kolom yang nanti dimasukan itu formatnya (yyyy-mm-dd))
    df['mths_since_' + column] = round(pd.to_numeric((today_date - df[column]) / np.timedelta64(1, 'D')/ 30.436875)) #<---fungsi ini berfungsi 'hari ini' - 'tanggal sesuai kolom' / perbulan.
    df.drop(columns = [column], inplace=True) #<--- ini ngehapus data kolom yang dimasukan, karena kan udh ada kolom mths_since

# apply to X_train
date_columns(X_train, 'earliest_cr_line')
date_columns(X_train, 'issue_d')
date_columns(X_train, 'last_pymnt_d')
date_columns(X_train, 'last_credit_pull_d')




<ipython-input-169-e0b8c307590b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_datetime(df[column], format = "%Y-%m-%d") #<---mendefinisikan kolom yang nanti dimasukan itu formatnya (yyyy-mm-dd))
<ipython-input-169-e0b8c307590b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mths_since_' + column] = round(pd.to_numeric((today_date - df[column]) / np.timedelta64(1, 'D')/ 30.436875)) #<---fungsi ini berfungsi 'hari ini' - 'tanggal sesuai kolom' / perbulan.
<ipython-input-169-e0b8c3

In [ ]:
X_train.head()

,term,emp_length,mths_since_earliest_cr_line,mths_since_issue_d,mths_since_last_pymnt_d,mths_since_last_credit_pull_d
345211,36,5,212.0,96.0,94.0,88.0
741379,36,1,205.0,92.0,85.0,85.0
1248925,36,10,288.0,99.0,92.0,85.0
610328,36,6,265.0,106.0,101.0,85.0
187906,36,5,189.0,89.0,NaN,85.0


In [ ]:
# kita lakukan pada data X_test
date_columns(X_test, 'earliest_cr_line')
date_columns(X_test, 'issue_d')
date_columns(X_test, 'last_pymnt_d')
date_columns(X_test, 'last_credit_pull_d')

<ipython-input-169-e0b8c307590b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_datetime(df[column], format = "%Y-%m-%d") #<---mendefinisikan kolom yang nanti dimasukan itu formatnya (yyyy-mm-dd))
<ipython-input-169-e0b8c307590b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mths_since_' + column] = round(pd.to_numeric((today_date - df[column]) / np.timedelta64(1, 'D')/ 30.436875)) #<---fungsi ini berfungsi 'hari ini' - 'tanggal sesuai kolom' / perbulan.
<ipython-input-169-e0b8c3

In [ ]:
X_test.head()

,term,emp_length,mths_since_earliest_cr_line,mths_since_issue_d,mths_since_last_pymnt_d,mths_since_last_credit_pull_d
127029,36,2,241.0,101.0,89.0,85.0
236567,36,1,316.0,101.0,87.0,85.0
183902,36,0,287.0,102.0,92.0,86.0
126091,36,6,245.0,101.0,99.0,94.0
1335086,60,10,353.0,107.0,85.0,85.0


okay sip, sekarang kita cek kembali Missing Value nya, kali aja ada

In [ ]:
X_train.isnull().sum()

,0
term,0
emp_length,0
mths_since_earliest_cr_line,0
mths_since_issue_d,0
mths_since_last_pymnt_d,9072
mths_since_last_credit_pull_d,26


In [ ]:
X_test.isnull().sum()

,0
term,0
emp_length,0
mths_since_earliest_cr_line,0
mths_since_issue_d,0
mths_since_last_pymnt_d,2304
mths_since_last_credit_pull_d,3


In [ ]:
# kita isi aja pakai median
X_train.fillna(X_train.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)


<ipython-input-175-2f996d4a3ed8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.median(), inplace=True)
<ipython-input-175-2f996d4a3ed8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_test.median(), inplace=True)


In [ ]:
X_train.shape, X_test.shape

((1104344, 6), (276087, 6))

##Modelling
untuk persoalan kita kali ini, yaitu 'Binary Classification', kita akan coba dulu dengan model yang paling sederhana, yaitu logistic regression. baru setelah dicoba dan ternyata kurang memuaskan, kita coba tambah datanya lagi atau ganti pakai model yang lebih kompleks. jadi, aturannya adalah pakai dulu model paling ssederhana, kalau kurang bagus, baru ke yang lebih kompleks seperti Deission Tree atau Random Forest

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
result = pd.DataFrame(list(zip(y_pred,y_test)), columns = ['y_pred', 'y_test'])
result.head()

,y_pred,y_test
0,1,1
1,0,1
2,1,1
3,1,1
4,0,0


dari list diatas, prediksi kita meleset satu, pada index 1, di prediksi 0, tapi di jawabannya (y_test) nya adalah 1

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8265256966101265

##Cek Evaluasi Model

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.91      0.84    140575
           1       0.89      0.74      0.81    135512

    accuracy                           0.83    276087
   macro avg       0.84      0.83      0.82    276087
weighted avg       0.83      0.83      0.83    276087



In [ ]:
# kita coba liat pemerataan data nya
y_train.value_counts(normalize=True)

,proportion
good_bad,
0,0.509171
1,0.490829


berdasarkan data diatas, nilai 0 (good loan) nya sebanyak 50%% dan nilai 1 (bad loan) nya sebanyak 49%. ini menandakan kalau dalam kasus ini, sudah tidak terjadi imbalance class (kelas yang tidak seimbang), karena pada sebelumnya sudah diseimbangkan antara nilai 0 dan 1 (pada feature good_bad)

kita coba cek skor secara keseluruhan, maksudnya selain accuracy (precision, recall, f1-score)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.91      0.84    140575
           1       0.89      0.74      0.81    135512

    accuracy                           0.83    276087
   macro avg       0.84      0.83      0.82    276087
weighted avg       0.83      0.83      0.83    276087



jika diperhatikan, sudah tidak ada ketimpangan yang signifikan antara nilai 0 dan 1 pada setiap macam evaluasi model (mungkin hanya recall, 0 nya 91, 1 nya 74).

maka dari itu, model bisa dikatakan baik

In [ ]:
# kita akan pakai mode coef untuk menginterretasikannya
model.coef_

array([[ 0.01277738, -0.02900513, -0.00065616,  0.03360602,  0.47086017,
        -0.40251248]])

In [ ]:
df_coeff = pd.DataFrame(model.coef_, columns=X_train.columns)
df_coeff

,term,emp_length,mths_since_earliest_cr_line,mths_since_issue_d,mths_since_last_pymnt_d,mths_since_last_credit_pull_d
0,0.012777,-0.029005,-0.000656,0.033606,0.47086,-0.402512


sebelumnya, kita jabarkan dulu mengenai fitur fiturnya

term: Durasi pinjaman.

emp_length: Lama masa kerja.

mths_since_earliest_cr_line: Jumlah bulan sejak rekening kredit pertama dibuka.

mths_since_issue_d: menunjukan sejak berapa bulan  duitnya dicairkan

mths_since_last_pymnt_d: Jumlah bulan sejak pembayaran terakhir.

mths_since_last_credit_pull_d: Jumlah bulan sejak laporan kredit terakhir ditarik.

ketika bernilai positif, maka loan tersebut cenderung akan gagal bayar, dan jika nilai negatif, maka cenderung tidak akan gagal bayar

mari kita bahasakan.
* untuk yang kolom term (durasi pinjaman), semakin tinggi term (jangka waktu (dalam satuan bulan) dalam melunaskan utang), maka semakin tinggi juga probabilitas kecenderungan gagal bayar (cenderung galbay)
* untuk emp_length (lama masa kerja nasabah tersebut diekerjaannya), semakin lama dia bekerja, krna disitu negatif, maka menunjukan bahwa nasabah dgn emp_length yang tinggi akan cenderung membayar hutang, karena berarti pekerjaannya stabil sehingga mampu untuk melunasi hutangnya.
* untuk mths_since_earliest_cr_line, semakin lama nasabah tersebut sudah tergabung, maka kemungkinannya nasabah tersebut akan membayar atau tidak akan galbay (ditandai dgn nilai negatif)
* untuk mths_since_issue_d, yang menunjukan sejak berapa bulan  duitnya dicairkan, maka semakin besar bulannya (artinya semakin sudah lama nasabah tersebut berhutang), maka nasabah tersebut cenderung galbay (ditandai dgn nilai yang positif dan termasuk tinggi)
* untuk mths_since_last_pymnt_d, Jumlah bulan sejak pembayaran terakhir, dimana semakin besar jumlahh bulan sejak nasabah tersebut membayar cicilannya (yang artinya nunggak), maka semakin besar juga kecenderungan nasabah tersebut gagalbayar. karena diperkirakan nasabah tersebut akan menunggak lagi nantinya.

